In [1]:
import tensorflow as tf
import collections
import numpy as np

In [27]:
with open("/Users/pim/Downloads/enwik9tok.short") as f:
    data = f.read().split(" ")
    counter = collections.Counter(data)
    count_pairs = sorted(counter.items(), key=lambda x: (-x[1], x[0]))
    words, _ = list(zip(*count_pairs))
    word_to_id = dict(zip(words, range(len(words))))
    tokens = np.array([word_to_id[w] for w in data])
    vocab_size = len(words)

In [28]:
batch_size = 25
embedding_size = 5
hidden_size = 10
num_steps = 4
dt = tf.float32

def get_batch():
    begins = np.random.randint(vocab_size - num_steps - 1, size=batch_size)[:,np.newaxis]
    ranges = np.arange(num_steps, dtype=np.int)[np.newaxis,:]
    indices = begins + ranges
    batch_inputs = tokens[indices]
    return batch_inputs, tokens[begins+1].reshape([-1])

In [58]:
with tf.Graph().as_default(), tf.Session() as sess:
    input_placeholder = tf.placeholder(tf.int32, shape=(batch_size, num_steps))
    embedding = tf.get_variable("embedding", [vocab_size, embedding_size], dtype=dt)
    input_vectors = tf.nn.embedding_lookup(embedding, input_placeholder)

    cell = tf.nn.rnn_cell.BasicRNNCell(num_units=hidden_size)
    initial_state = cell.zero_state(batch_size, dt)

    inputs = [tf.squeeze(input_step, [1])
              for input_step in tf.split(1, num_steps, input_vectors)]
    outputs, state = tf.nn.rnn(cell, inputs, initial_state=initial_state)
    
    output  = outputs[-1]
    sigmoid = tf.sigmoid(output)
    weights = tf.Variable(tf.truncated_normal([hidden_size, embedding_size], stddev=.001))
    bias    = tf.Variable(tf.zeros([embedding_size]))
    activation = tf.matmul(sigmoid, weights) + bias
    
    target_placeholder = tf.placeholder(tf.int32, shape=[batch_size])
    
    init = tf.initialize_all_variables()
    sess.run(init)
    
    
    batch_inputs, batch_targets = get_batch()
    o = sess.run(activation, feed_dict={
        input_placeholder: batch_inputs,
        target_placeholder: batch_targets
    })
    print(o.shape)

(25, 5)
